In [1]:
%%capture
import pickle
import numpy as np
import math
import scipy as sp
from EM import EM
from plot import plot
from preprocess import preprocess
#from EM_individual_params import EM 

### Modeling choices (which generates the lowest MSE and the plots in write up)
* Cutoff (only consider the data point with at least certain number of observations): 5 --> 2214 data points, 215 bins
* Bin size (length of time interval used to bin the time series): 18 hrs
* Parameters initialization
    * Both treatment and static coefficients: Gaussian with mean 0.1 plus small white noise
* Missingness
    * After binning the data, only consider the data points with less than 40% missing observations
* Chronic conditions and age
    * Set to zero

Coefficients learned with best population level results
* Coefficient for nsaid: [0.31291354 0.11150224 0.02927106]
* Coefficient for transfusion_plasma: [0.21646024 0.12142273 0.07316288]
* Coefficient for transfusion_platelet: [-0.24024057 -0.41378184 -0.28599656]
* Coefficient for anticoagulant: [-0.04605939  0.15568198  0.06412978]
* Coefficient for aspirin: [0.25202486 0.1102651  0.03499285]

Coefficients (jointly) learned with best population level results
* Coefficient for nsaid: [ 0.09144572  0.01808053 -0.00736706]
* Coefficient for transfusion_plasma: [-0.29370552 -0.15658492 -0.12820925]
* Coefficient for transfusion_platelet: [-0.32318352 -0.44511952 -0.43787362]
* Coefficient for anticoagulant: [-0.46557922  0.00142221 -0.02525039]
* Coefficient for aspirin: [-0.04017403 -0.04024049 -0.00733096]
* Coefficient for chronic kidney failure: 0.27480724324859296
* Coefficient for sickle cell: 0.238516267304465
* Coefficient for age: 0.01752724784756275

In [2]:
# Preprocess Setting
bin_size = 60 * 18
cutoff = 5
missing_pct = .4
c_zero = False

# EM Setting
num_past_effects = 3
training_pct = .8
single_effect = False

In [3]:
data = pickle.load(open('../Data/unimputed_inr_patient_data.pkl', 'rb'), encoding='latin1')

In [4]:
y, X, c = preprocess(data, cutoff, bin_size, missing_pct=missing_pct, c_zero=c_zero)

In [5]:
y.shape

(2214, 215)

### Reasons for Errors
* Assume the a single set of parameters generate all the data. This assumption could be violated since:
    1. Each patient respond to treatment differently
    2. Different treatment and chronic conditions may have various interaction effects on different individuals
* After changing treatment coefficients to be on individual level, the average coefficients are similar to the case of population level coefficients. The fit on some plots improve slightly but not much.  
* Assume observed value is well explained by the treatment and chronic variable. whereas in reality there might be other variable that contributes to the observed
* Assume linear transition, emission and treatment/chronic effects
* Seeing from the data, treatments don't seem to show clear effects on INR, from plots and from calculations.

In [6]:
em = EM(y, X, c, num_past_effects, 0, train_pct=training_pct, single_effect=single_effect)

In [7]:
%%time
em.run_EM(2000)

observed log likelihood -26210.533502075657
observed log likelihood -23063.251889157887
observed log likelihood -20319.88483316796
observed log likelihood -19045.123058261783
observed log likelihood -18453.585019539263
observed log likelihood -18150.483845620256
observed log likelihood -17967.37200314391
observed log likelihood -17836.224504814632
observed log likelihood -17730.117968150225
observed log likelihood -17638.552937642125
observed log likelihood -17557.544554260214
observed log likelihood -17485.702804662953
observed log likelihood -17422.699777572205
observed log likelihood -17368.675644065443
observed log likelihood -17324.00632780448
observed log likelihood -17289.20206011795
observed log likelihood -17264.840423588892
observed log likelihood -17251.488929546027
observed log likelihood -17249.595526316356
observed log likelihood -17259.3454354611
observed log likelihood -17280.507790476946
observed log likelihood -17312.31971124652
observed log likelihood -17353.46209267

KeyboardInterrupt: 

In [8]:
em.get_MSE()

0.7990421212083499

In [9]:
treatment_types = ['nsaid', 'transfusion_plasma', 'transfusion_platelet', 'anticoagulant', 'aspirin']
for i, treatment in enumerate(treatment_types):
    print('Coefficient for {}: {}'.format(treatment, em.A[:, i]))
static_types = ['chronic kidney failure', 'sickle cell', 'age']
for j, static in enumerate(static_types):
    print('Coefficient for {}: {}'.format(static, em.b[j]))

Coefficient for nsaid: [ 0.00718657  0.03553718 -0.01169237]
Coefficient for transfusion_plasma: [-0.42921708 -0.19435377 -0.22677482]
Coefficient for transfusion_platelet: [-0.53478138 -0.73497    -0.65893388]
Coefficient for anticoagulant: [-0.67991779 -0.0545573  -0.07365301]
Coefficient for aspirin: [-0.0202334  -0.01086478  0.06325083]
Coefficient for chronic kidney failure: -0.24463030424318313
Coefficient for sickle cell: -0.5140825186553146
Coefficient for age: -0.020434014971080817


In [10]:
random_patients = np.random.randint(0, y.shape[0], 0)
for i in random_patients:
    plot(em, i, bin_size=bin_size)